In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import spectools.lpsd as lpsd
import spectools.aux as aux
from pytdi.dsp import timeshift
from multiprocessing import Pool
import scipy.signal.windows as win

In [ ]:
# Noise generation
N = int(1e6) # samples
fs = 1e6 # Hertz
T = N/fs # seconds
delay = 1e-4 # seconds
print(f"Total time = {T} seconds")
print(f"Time delay = {delay} seconds")

y0 = np.random.normal(0, 1, N)
y1 = timeshift(y0, delay*fs) + 0.0*np.random.normal(0, 1, N)

n_trunc = int(2*delay*fs)

if n_trunc > 0:
    y0 = y0[n_trunc:-n_trunc]
    y1 = y1[n_trunc:-n_trunc]

print(f"Total length of time series after cropping = {len(y1)}")

coh_NBE = []
coh_dev = []
tf_NBE = []

l_axis = np.logspace(np.log10(delay*fs), np.log10(len(y1)), 100).astype(int)
for l in l_axis:
    tf0 = lpsd.ltf_single_bin([y0,y0], fs=fs, freq=fs/2, L=l, win=win.boxcar, olap=0.9)
    tf1 = lpsd.ltf_single_bin([y0,y1], fs=fs, freq=fs/2, L=l, win=win.boxcar, olap=0.9)
    coh_NBE.append((np.array(tf1.coh) - np.array(tf0.coh))/np.array(tf0.coh))
    coh_dev.append(tf1.coh_dev)
    tf_NBE.append((np.abs(tf1.Hxy) - np.abs(tf0.Hxy))/np.abs(tf0.Hxy))

In [ ]:
fig, ax = plt.subplots(figsize=(4,4), dpi=150)

ax.plot(delay/np.array(l_axis/fs), tf_NBE, c='k', ls='-', label=r'$\gamma_{xy}^2$')
ax.plot(delay/np.array(l_axis/fs), coh_NBE, c='k', ls='--', label=r'$|H_{xy}|$')

ax.set_xlabel(r'$\tau_0/T$')
ax.set_ylabel(r'Normalized bias error')
ax.set_xlim(0,1)
ax.set_ylim(-1,0)
ax.legend(loc='best', edgecolor='black', fancybox=True, shadow=True, framealpha=1, handlelength=2.5)
plt.show()